In [3]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

In [4]:
import fastf1 as ff1


In [ ]:
# Habilitar caché en una carpeta del proyecto
ff1.Cache.enable_cache("./cache")

NotADirectoryError: Cache directory does not exist! Please check for typos or create it first.

Division de los Datos

Vamos a definir a continuacion que features del dataset podrian ser utiles para entrenar nuestro modelo. Con este analisis pretendemos hacer una primera division entre features probablemente utiles y features peligrosas que en principio descartariamos para no generar dataleakage

Empecemos por cargar la carrera de Monza 2025 (Italian Grand Prix)

In [6]:
year = 2025
gp_name = "Italian Grand Prix"   # Monza
session_name = "R"               # Race

session = ff1.get_session(year, gp_name, session_name)
session.load()

laps = session.laps


req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) C:\Users\Lucas\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track

Veamos como esta compuesto el dataset:

In [14]:
print("Datos disponibles por cada vuelta:\n")
# Mostrar prolijamente las columnas
for col in laps.columns: 
    print(col)
    

Datos disponibles por cada vuelta:

Time
Driver
DriverNumber
LapTime
LapNumber
Stint
PitOutTime
PitInTime
Sector1Time
Sector2Time
Sector3Time
Sector1SessionTime
Sector2SessionTime
Sector3SessionTime
SpeedI1
SpeedI2
SpeedFL
SpeedST
IsPersonalBest
Compound
TyreLife
FreshTyre
Team
LapStartTime
LapStartDate
TrackStatus
Position
Deleted
DeletedReason
FastF1Generated
IsAccurate


Viendo la lista de datos:
- Laptime es nuestro Target

Podemos descartar rapidamente para evitar dataleakeage:
- Sector1Time, Sector2Time y Sector3Time ya que no son mas que el Target descompuesto en 3 tiempos.
- IsPersonalBest tambien la podemos descartar ya que usa info de todas las vueltas del piloto y depende de saber cuales fueron las mas rapidas

Por ser tiempos absolutos de sesión (los podriamos usar más adelante como features derivados, pero no crudos):
- Time
- Sector1SessionTime, Sector2SessionTime, Sector3SessionTime
- LapStartTime, LapStartDate
- PitInTime, PitOutTime

Por depender de telemetría dentro de la vuelta y qyue no es conocida antes de correrla:
- SpeedI1, SpeedI2, SpeedFL, SpeedST (speed traps durante la vuelta).

Por ser meta-datos o cosas mas raras:
- Deleted, DeletedReason, IsAccurate (sirven para filtrar vueltas malas, no para predecir)
- TyresNotChanged, LapFlags, LapCountTime, StartLaps, Outlap, etc. (campos poco claros / sin documentación fuerte, los ignoramos para empezar)


In [ ]:
# Me las guardo para despues
FEATURES_DROP_SIM = [
    "LapTime",
    "Sector1Time", "Sector2Time", "Sector3Time",
    "Sector1SessionTime", "Sector2SessionTime", "Sector3SessionTime",
    "Time", "LapStartTime", "LapStartDate",
    "PitInTime", "PitOutTime",
    "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST",
    "IsPersonalBest",
    "Deleted", "DeletedReason", "IsAccurate",
    "TyresNotChanged", "LapFlags", "LapCountTime", "StartLaps", "Outlap",
]


Filtro solo por las vueltas de Colapinto

In [9]:
# 2) Filtrar las vueltas de Franco Colapinto
driver_code = "COL"

col_laps = laps[
    (laps["Driver"] == driver_code)
    & laps["LapTime"].notna()
    & (~laps.get("Deleted", False))   # si 'Deleted' existe y es True, la sacamos
]

# Ordenamos por número de vuelta por prolijidad
col_laps = col_laps.sort_values("LapNumber").reset_index(drop=True)

# 3) Vector Y: tiempos de vuelta (en segundos)
Y = col_laps["LapTime"].dt.total_seconds().to_numpy()

print(f"Nº de vueltas de Colapinto en carrera: {len(Y)}")
print("Primeros 5 tiempos (s):", Y[:5])


Nº de vueltas de Colapinto en carrera: 49
Primeros 5 tiempos (s): [94.322 86.775 85.346 85.04  84.945]


Ahora como primera aproximacion basica voy a usar los siguientes features para mi X:

Datos de vuelta:
- LapNumber: Índice de vuelta en carrera → captura efecto de peso de combustible que baja, evolución de pista, etc.
- Stint: Número de stint → correlaciona con cambio de compuesto / momento de la carrera.

Datos de Neumáticos:
- Compound (categórica: SOFT / MEDIUM / HARD / INTERMEDIATE / WET)
- TyreLife (float: vueltas que tiene ese neumático)
- FreshTyre (bool: si el set era nuevo al inicio del stint)


Datos de Contexto de carrera:
- TrackStatus (string con códigos de estado de pista: verde, SC, VSC, etc.)
- Position (posición en carrera al final de la vuelta; para el simulador la vamos a aproximar como posición al inicio de la siguiente vuelta).


In [10]:
FEATURES_BASIC = [
    "LapNumber",
    "Stint",
    "Compound",
    "TyreLife",
    "FreshTyre",
    "TrackStatus",
    "Position",
]

# Nos quedamos solo con las que efectivamente existen en el DF por si faltara alguna
FEATURES_BASIC = [c for c in FEATURES_BASIC if c in col_laps.columns]

X = col_laps[FEATURES_BASIC].copy()
X.head()


,LapNumber,Stint,Compound,TyreLife,FreshTyre,TrackStatus,Position
0,1.0,1.0,MEDIUM,1.0,True,1,17.0
1,2.0,1.0,MEDIUM,2.0,True,1,17.0
2,3.0,1.0,MEDIUM,3.0,True,1,17.0
3,4.0,1.0,MEDIUM,4.0,True,1,17.0
4,5.0,1.0,MEDIUM,5.0,True,1,17.0
